In [ ]:
pip install requests bs4 pandas firebase-admin

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

Initialize Firestore DB

In [26]:
#load in firestore credentials
# https://firebase.google.com/docs/firestore/quickstart
# DON"T FORGET TO DELETE THE PATH WHEN PUSHING TO GITHUB!!!!
cred = credentials.Certificate('PATH_TO_FIRESTORE_KEY')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

This is what I would be storing for each twit

In [27]:
twit_url = "https://stocktwits.com/message/497084294"
print('url: ', twit_url)
page = requests.get(twit_url)
soup = BeautifulSoup(page.content, 'html.parser')
twit_text = str(soup.head.find("meta", attrs={"name" : "description"})["content"])
print('text: ', twit_text)
twit_user = str(soup.head.find("meta", attrs={"name" : "twitter:title"})["content"]).replace(' | Stocktwits','')
print('user: ', twit_user)
print('userUrl: https://stocktwits.com/'+twit_user.replace('@',''))

url:  https://stocktwits.com/message/497084294
text:  $SAVA stop loss raid this afternoon was great buying opportunity loaded shares under $30!
user:  @CaymanSlushFund
userUrl: https://stocktwits.com/CaymanSlushFund


Example on how to get data - get our index flag for last message scraped

In [34]:
# we would first start by retrieving the last good message id
doc_ref = db.collection(u'twit').document(u'startIndex')
# https://firebase.google.com/docs/firestore/query-data/get-data
doc = doc_ref.get()
if doc.exists:
    print(f'Document data: {doc.to_dict()}')
else:
    print(u'No such document!')

Document data: {'testIndex': 1, 'index': 0}


Example on how to push data - updating our index flag

In [35]:
data = {
    u'testIndex': 10,
}
# https://firebase.google.com/docs/firestore/manage-data/add-data
# Updating our doc_ref from above to whatever we specify in data.index
doc_ref.set(data, merge=True)
# Check if it worked by replaying above get data example and see if it changed

update_time {
  seconds: 1668814840
  nanos: 26139000
}

Next steps for creating the actual scraper:
1. Loop through starting at index
2. https://stocktwits.com/message/# -> # should be the index
3. Use bs4 and add current data to csv as new row
4. End loop when no more new messages ~ will need to talk with team on how to determine this
5. Update index to what first bad index was
6. Push csv to storage - I will add this code in soon
7. End